# 패키지 및 데이터 불러오기

In [1]:
!pip install sentence-transformers

from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
import torch
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np
import re
from transformers import GPT2Tokenizer, GPT2Model, GPT2LMHeadModel
from sentence_transformers import SentenceTransformer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 6.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [17]:
# 데이터셋 로드
interests_df = pd.read_csv('/content/크롤링_전처리.csv')
courses_df = pd.read_csv('/content/merged_lecture_3.csv')

print(interests_df.head())
print(courses_df.head())

         검색어  출처                                    제목  \
0  중장년 취미 생활  뉴스              성북구 중장 1 인 가구 힐링 원데이 클래스   
1  중장년 취미 생활  뉴스      성북구 중장 1 인 가구 셀프 집 수리 힐링 원데이 클래스   
2  중장년 취미 생활  뉴스                   성북구 중장 1 인 가구 맞춤 야간   
3  중장년 취미 생활  뉴스  quot 미래 일단 잡자 quot MZ 세대 겨냥 이색 보험 뜬다   
4  중장년 취미 생활  뉴스          전북 비정규직 인원 3만 명대 육박 프리터 족 급증   

                                                  설명  \
0  향후 성북구 고려대학교 KU 건강 연구소 해 우울 감 을 경험 하는 중장 집단 심리...   
1  성북구 향후 고려대학교 KU 건강 연구소 해 우울 감 을 경험 하는 중장 집단 심리...   
2  내년 에는 동아리 을 하는 1 인 가구 생활 뿐 만 건강 까지 챙긴다는 이다 이승 ...   
3  윤씨 자산 중장 년층 비해 부족하지만 하고 싶은 일 많은 20 대들을 위 보험 을 ...   
4  회사 다닐 보다 시간 구애 받지 않고 자유로운 여가 생활 을 즐기는 여유 많아져 관...   

                                                  링크  
0  https://n.news.naver.com/mnews/article/081/000...  
1  https://n.news.naver.com/mnews/article/421/000...  
2  https://n.news.naver.com/mnews/article/629/000...  
3  https://www.greened.kr/news/articleView.html?i...  
4  http://www.domin.co.kr/news/articleV

# BERT 모델

In [3]:
# BERT 모델 로드
model = SentenceTransformer('paraphrase-mpnet-base-v2')

# 텍스트 전처리 함수
def preprocess(text):
    if isinstance(text, str):
        text = text.lower()
        text = re.sub(r'[^a-zA-Z0-9가-힣\s]', '', text)
    else:
        text = ''
    return text

# 데이터프레임 전처리
interests_df['제목'] = interests_df['제목'].apply(preprocess)
interests_df['설명'] = interests_df['설명'].apply(preprocess)
courses_df['강의 명'] = courses_df['강의 명'].apply(preprocess)
courses_df['강의 카테고리명'] = courses_df['강의 카테고리명'].apply(preprocess)

# 관심사 임베딩 생성
interest_embeddings = model.encode((interests_df['제목'].astype(str) + " " + interests_df['설명'].astype(str)).tolist())

# 강의명 임베딩 생성
course_embeddings = model.encode((courses_df['강의 명'] + " " + courses_df['강의 카테고리명']).tolist())

# 유사도 매트릭스 계산 (기존 관심사와 강의명 매칭)
similarity_matrix = cosine_similarity(interest_embeddings, course_embeddings)

# 가중치 적용 임베딩 계산 함수
def calculate_weighted_embedding(interests, weights):
    input_embeddings = model.encode(interests)
    weighted_embedding = np.average(input_embeddings, axis=0, weights=weights)
    return weighted_embedding

# 유사도 계산 함수
def calculate_similarity(embedding, embeddings):
    return cosine_similarity([embedding], embeddings)

# 강의 추천 함수
def recommend_courses(interests, weights, similarity_matrix, top_n=3):
    # 입력된 관심사의 가중치 적용 임베딩 계산
    weighted_input_embedding = calculate_weighted_embedding(interests, weights)

    # 입력된 관심사와 기존 관심사 간의 유사도 계산
    interest_similarity = calculate_similarity(weighted_input_embedding, interest_embeddings)

    # 유사도가 높은 기존 관심사 인덱스 찾기
    best_match_index = np.argmax(interest_similarity)

    # 해당 관심사와 유사한 강의 찾기
    related_courses_similarity = similarity_matrix[best_match_index]

    # 유사도가 높은 순으로 강의명 정렬
    sorted_indices = np.argsort(related_courses_similarity)[::-1]

    # 상위 n개의 강의명 반환
    recommended_courses = courses_df['강의 명'].iloc[sorted_indices[:top_n]]

    return recommended_courses.tolist()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [21]:
# 예시 사용법

interests = ["marketing", "Entrepreneurship"]
weights = [100, 20]
print(recommend_courses(interests, weights, similarity_matrix, top_n=3))

['[세일즈 뉴노멀] Value Selling을 통한 거래 확대, 핵심 거래처 관리 전략', '박문각 공인중개사 1차 - 문제풀이 - 민법 및 민사특별법', '박문각 공인중개사 1차 - 기본이론 - 민법 및 민사특별법']
